In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "xxx"

In [2]:
!pip install -q langchain-google-genai langchain-core requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [4]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [5]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [6]:
multiply.name

'multiply'

In [7]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [8]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [9]:
# tool binding

In [10]:
llm = ChatGoogleGenerativeAI(model='gemini-1.5-pro')

In [11]:
llm.invoke('hi')

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-8897ded9-a047-4ec7-a0d4-0ce9037d3ee2-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [12]:
llm_with_tools = llm.bind_tools([multiply])

In [13]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="I'm doing well, thank you for asking. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-0bf346a9-f461-471b-8242-739f321a7dd1-0', usage_metadata={'input_tokens': 24, 'output_tokens': 19, 'total_tokens': 43, 'input_token_details': {'cache_read': 0}})

In [14]:
query = HumanMessage('can you multiply 3 with 1000')

In [15]:
messages = [query]

In [16]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [17]:
result = llm_with_tools.invoke(messages)

In [18]:
messages.append(result)

In [19]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 1000.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-b46c155a-2333-461e-bbef-8b2e339f3a50-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 1000.0}, 'id': '6e12ac25-8094-40d4-9e49-9634797d0fe0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 31, 'output_tokens': 5, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}})]

In [20]:
tool_result = multiply.invoke(result.tool_calls[0])

In [21]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='6e12ac25-8094-40d4-9e49-9634797d0fe0')

In [22]:
messages.append(tool_result)

In [23]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 1000.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-b46c155a-2333-461e-bbef-8b2e339f3a50-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 1000.0}, 'id': '6e12ac25-8094-40d4-9e49-9634797d0fe0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 31, 'output_tokens': 5, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='3000', name='multiply', tool_call_id='6e12ac25-8094-40d4-9e49-9634797d0fe0')]

In [24]:
llm_with_tools.invoke(messages).content

'3 multiplied by 1000 is 3000.'

In [ ]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/56adad3742581/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [26]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [27]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1746403201,
 'time_last_update_utc': 'Mon, 05 May 2025 00:00:01 +0000',
 'time_next_update_unix': 1746489601,
 'time_next_update_utc': 'Tue, 06 May 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 84.5939}

In [28]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [29]:
# tool binding
llm = ChatGoogleGenerativeAI(model='gemini-1.5-pro')

In [30]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [31]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [32]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [33]:
ai_message = llm_with_tools.invoke(messages)

In [35]:
messages.append(ai_message)

In [36]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'USD', 'base_currency': 'INR'},
  'id': 'ad1ea625-31ca-40cb-9b85-d2c4f6abcaa3',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10.0},
  'id': '91129b4b-6f9b-435b-b737-c2d7697e83fe',
  'type': 'tool_call'}]

In [37]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [38]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'convert', 'arguments': '{"base_currency_value": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-b0c29e0d-b1f3-46db-a885-149221c2d9c0-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'USD', 'base_currency': 'INR'}, 'id': 'ad1ea625-31ca-40cb-9b85-d2c4f6abcaa3', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': 10.0, 'conversion_rate': 0.01182}, 'id': '91129b4b-6f9b-435b-b737-c2d7697e83fe', 'type': 'tool_call'}], usage_metadata={'input_tokens': 92, 'output_tokens': 20, 'total_tokens': 112, 'input_token_details': {'cache_read': 0}}),
 AIMessage(content='', additional_kwar

In [39]:
llm_with_tools.invoke(messages).content

'The conversion factor between INR and USD is 0.01182. Based on this, 10 INR is equal to 0.1182 USD.'

In [40]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

<ipython-input-40-23d1b63d4a11>:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [41]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Thought:I am doing well, thank you for asking. I should ask how the user is doing as well.
Action:
```json
{
  "action": "Final Answer",
  "action_input": "I'm doing well, thank you for asking. How are you?"
}
```

> Finished chain.
